# Predict Future Sales

## Introduction

$Description$: ...

### Libraries

In [1]:
import pandas as pd
import numpy as np
from numpy import char
from matplotlib import pyplot as plt
import seaborn as sns
import datetime

In [2]:
item_categories = pd.read_csv("../data/item_categories.csv")
items = pd.read_csv("../data/items.csv")
df_train = pd.read_csv("../data/sales_train.csv")
df_sub = pd.read_csv("../data/sample_submission.csv")
shops = pd.read_csv("../data/shops.csv")
df_test = pd.read_csv("../data/test.csv")

## Exploratory Data Analysis (EDA)

In [75]:
datasets = [('ITEMS CATEGORIES', item_categories), ('ITEMS', items), ('TRAIN DATA', df_train), ('SHOPS', shops), ('TEST DATA', df_test)]

for dn, d in datasets:
    print(dn)
    print("Shape of dataset {}".format(d.shape))
    print()
    print("Columns of dataset \n{}".format(d.dtypes))
    print()
    print()

ITEMS CATEGORIES
Shape of dataset (84, 2)

Columns of dataset 
item_category_name    object
item_category_id       int64
dtype: object


ITEMS
Shape of dataset (22170, 3)

Columns of dataset 
item_name           object
item_id              int64
item_category_id     int64
dtype: object


TRAIN DATA
Shape of dataset (2935849, 6)

Columns of dataset 
date               object
date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object


SHOPS
Shape of dataset (60, 2)

Columns of dataset 
shop_name    object
shop_id       int64
dtype: object


TEST DATA
Shape of dataset (214200, 3)

Columns of dataset 
ID         int64
shop_id    int64
item_id    int64
dtype: object




In [90]:
np.sort(shops.loc[:, 'shop_id'].unique())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59], dtype=int64)

In [101]:
df_test[df_test.shop_id.isin(df_train.loc[:, 'shop_id'].unique()) == False].shop_id.unique()

array([], dtype=int64)

In [81]:
shops.loc[:, 'shop_name'].unique()

array(['!Якутск Орджоникидзе, 56 фран', '!Якутск ТЦ "Центральный" фран',
       'Адыгея ТЦ "Мега"', 'Балашиха ТРК "Октябрь-Киномир"',
       'Волжский ТЦ "Волга Молл"', 'Вологда ТРЦ "Мармелад"',
       'Воронеж (Плехановская, 13)', 'Воронеж ТРЦ "Максимир"',
       'Воронеж ТРЦ Сити-Парк "Град"', 'Выездная Торговля',
       'Жуковский ул. Чкалова 39м?', 'Жуковский ул. Чкалова 39м²',
       'Интернет-магазин ЧС', 'Казань ТЦ "Бехетле"',
       'Казань ТЦ "ПаркХаус" II', 'Калуга ТРЦ "XXI век"',
       'Коломна ТЦ "Рио"', 'Красноярск ТЦ "Взлетка Плаза"',
       'Красноярск ТЦ "Июнь"', 'Курск ТЦ "Пушкинский"',
       'Москва "Распродажа"', 'Москва МТРЦ "Афи Молл"',
       'Москва Магазин С21', 'Москва ТК "Буденовский" (пав.А2)',
       'Москва ТК "Буденовский" (пав.К7)', 'Москва ТРК "Атриум"',
       'Москва ТЦ "Ареал" (Беляево)', 'Москва ТЦ "МЕГА Белая Дача II"',
       'Москва ТЦ "МЕГА Теплый Стан" II',
       'Москва ТЦ "Новый век" (Новокосино)', 'Москва ТЦ "Перловский"',
       'Москва Т

In [119]:
shops_name = shops.shop_name.str.split(" ", expand=True).rename(columns = lambda x: f"Part_Name_{x+1}")
shops_name.drop(['Part_Name_3', 'Part_Name_4', 'Part_Name_5', 'Part_Name_6'], axis=1, inplace=True)
shops = pd.merge(shops, shops_name, left_index=True, right_index=True).rename(columns={'Part_Name_1':'city', 'Part_Name_2':'shop_attribute'})

In [130]:
item_categories_name = item_categories.item_category_name.str.split("-", expand=True).rename(columns = lambda x: f"Part_Name_{x+1}")
item_categories_name.Part_Name_3.unique()

array([None, 'Ray', 'Ray 3D', 'Ray 4K'], dtype=object)

In [131]:
item_categories.item_category_name.unique()

array(['PC - Гарнитуры/Наушники', 'Аксессуары - PS2', 'Аксессуары - PS3',
       'Аксессуары - PS4', 'Аксессуары - PSP', 'Аксессуары - PSVita',
       'Аксессуары - XBOX 360', 'Аксессуары - XBOX ONE', 'Билеты (Цифра)',
       'Доставка товара', 'Игровые консоли - PS2',
       'Игровые консоли - PS3', 'Игровые консоли - PS4',
       'Игровые консоли - PSP', 'Игровые консоли - PSVita',
       'Игровые консоли - XBOX 360', 'Игровые консоли - XBOX ONE',
       'Игровые консоли - Прочие', 'Игры - PS2', 'Игры - PS3',
       'Игры - PS4', 'Игры - PSP', 'Игры - PSVita', 'Игры - XBOX 360',
       'Игры - XBOX ONE', 'Игры - Аксессуары для игр',
       'Игры Android - Цифра', 'Игры MAC - Цифра',
       'Игры PC - Дополнительные издания',
       'Игры PC - Коллекционные издания', 'Игры PC - Стандартные издания',
       'Игры PC - Цифра', 'Карты оплаты (Кино, Музыка, Игры)',
       'Карты оплаты - Live!', 'Карты оплаты - Live! (Цифра)',
       'Карты оплаты - PSN', 'Карты оплаты - Windows (Цифра)',

In [103]:
shops_name

,Part_Name_1,Part_Name_2,Part_Name_3,Part_Name_4,Part_Name_5,Part_Name_6
0,!Якутск,"Орджоникидзе,",56,фран,None,None
1,!Якутск,ТЦ,"""Центральный""",фран,None,None
2,Адыгея,ТЦ,"""Мега""",None,None,None
3,Балашиха,ТРК,"""Октябрь-Киномир""",None,None,None
4,Волжский,ТЦ,"""Волга","Молл""",None,None
5,Вологда,ТРЦ,"""Мармелад""",None,None,None
6,Воронеж,"(Плехановская,",13),None,None,None
7,Воронеж,ТРЦ,"""Максимир""",None,None,None
8,Воронеж,ТРЦ,Сити-Парк,"""Град""",None,None
9,Выездная,Торговля,None,None,None,None


In [9]:
df_train[df_train.item_id == 22154]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
3270,23.01.2013,0,24,22154,999.0,1.0
17081,20.01.2013,0,27,22154,999.0,1.0
25918,02.01.2013,0,25,22154,999.0,1.0
25919,03.01.2013,0,25,22154,999.0,1.0
25920,20.01.2013,0,25,22154,999.0,1.0
25921,23.01.2013,0,25,22154,999.0,1.0
25922,26.01.2013,0,25,22154,999.0,1.0
35509,27.01.2013,0,6,22154,999.0,1.0
45824,10.01.2013,0,15,22154,999.0,1.0


In [7]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [8]:
df_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [20]:
df_train

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [3]:
df_train['year']=pd.DatetimeIndex(df_train['date']).year
df_train['month']=pd.DatetimeIndex(df_train['date']).month
df_train['day']=pd.DatetimeIndex(df_train['date']).day
df_train['month_year']=pd.to_datetime(df_train['date']).dt.to_period('M')

In [4]:
df_train.dtypes

date                 object
date_block_num        int64
shop_id               int64
item_id               int64
item_price          float64
item_cnt_day        float64
year                  int64
month                 int64
day                   int64
month_year        period[M]
dtype: object

In [79]:
df_train[df_train.date_block_num == 33]

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
2882335,23.10.2015,33,45,13315,649.0,1.0
2882336,05.10.2015,33,45,13880,229.0,1.0
2882337,02.10.2015,33,45,13881,659.0,1.0
2882338,12.10.2015,33,45,13881,659.0,1.0
2882339,04.10.2015,33,45,13923,169.0,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.0,1.0
2935845,09.10.2015,33,25,7460,299.0,1.0
2935846,14.10.2015,33,25,7459,349.0,1.0
2935847,22.10.2015,33,25,7440,299.0,1.0
